# Calculate

In [1]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

In [2]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [3]:
import sys
sys.path.insert(0,'../')

TODO
B2
solvation potential 
chi
wrappers for omega, h, etc

## CALCS

In [5]:
%%run_and_write ../pyPRISM/calculate/prism/pair_correlation.py
#!python
from __future__ import division,print_function
from pyPRISM.core.Space import Space

def pair_correlation(PRISM):
    '''Calculate the pair correlation function from a PRISM object
    
    After convergence, the stored total correlation function can 
    simply be shifted to obtain the pair-correlation function 
    i.e. the radial distribution function. 
    
    .. math::
        g(r) = h(r) + 1.0
        
    Parameters
    ----------
    PRISM: pyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    
    Returns
    -------
    pairCorr: pyPRISM.core.MatrixArray.MatrixArray
        The full MatrixArray of pair correlation functions.
    
    '''
    
    if PRISM.totalCorr.space == Space.Fourier:
        PRISM.domain.MatrixArray_to_real(PRISM.totalCorr)
    
    PRISM.pairCorr = PRISM.totalCorr + 1.0
    
    return PRISM.pairCorr

Overwriting ../pyPRISM/calculate/prism/pair_correlation.py


In [31]:
%%run_and_write ../pyPRISM/calculate/prism/structure_factor.py
#!python
from __future__ import division,print_function
from pyPRISM.core.Space import Space

def structure_factor(PRISM):
    '''Calculate the **dimensionless** structure factor from a PRISM object
    
    
    
    .. math::
        S(k) = \omega(k) + h(k)
        
    Parameters
    ----------
    PRISM: pyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    Returns
    -------
    structureFactor: pyPRISM.core.MatrixArray.MatrixArray
        The full MatrixArray of structure factors
    
    '''
    
    if PRISM.totalCorr.space == Space.Real:
        PRISM.domain.MatrixArray_to_fourier(PRISM.totalCorr)
        
    if PRISM.omega.space == Space.Real:
        PRISM.domain.MatrixArray_to_fourier(PRISM.omega)
    
    structureFactor = (PRISM.totalCorr*PRISM.pairDensityMatrix + PRISM.omega)/PRISM.siteDensityMatrix
    # structureFactor = PRISM.totalCorr + PRISM.omega
    
    return structureFactor

Overwriting ../pyPRISM/calculate/prism/structure_factor.py


In [8]:
%%run_and_write ../pyPRISM/calculate/prism/pmf.py
#!python
from __future__ import division,print_function
from pyPRISM.core.MatrixArray import MatrixArray
from pyPRISM.core.Space import Space
from pyPRISM.calculate.prism.pair_correlation import pair_correlation
import numpy as np

def pmf(PRISM):
    '''Calculate the potentials of mean force from a PRISM object
    
    .. math::
        W(r) = -k_{B} T log(h(r)+1.0)
        
    Parameters
    ----------
    PRISM: pyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    Returns
    -------
    pmf: pyPRISM.core.MatrixArray.MatrixArray
        The full MatrixArray of potentials of mean force
    
    '''
    rdf = pair_correlation(PRISM)
    
    #let's ignore any warnings about negative values in the log function
    with np.errstate(invalid='ignore'):
        rdf = -1.0 * PRISM.kT * np.log(rdf.data)
    
    #length and rank will be inferred from data
    pmf = MatrixArray(data=rdf,space=Space.Real,length=None,rank=None)
    
    return pmf

Overwriting ../pyPRISM/calculate/prism/pmf.py


In [4]:
%%run_and_write ../pyPRISM/calculate/prism/chi.py
#!python
from __future__ import division,print_function
from pyPRISM.core.MatrixArray import MatrixArray
from pyPRISM.core.Space import Space
import numpy as np

#XXX Perhaps this should return a table with sentinel None's along the diagonal?
def chi(PRISM):
    '''Calculate the wavenumber-dependent effective interaction parameter (chi)
    
    .. math::
        
        \chi(k)  = 0.5 * \rho * (\hat{C}_{AA}(k) + \hat{C}_{BB}(k) - 2* + \hat{C}_{AB}(k))
        
    Parameters
    ----------
    PRISM: pyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    Returns
    -------
    chi: dict
        Dictionary of all wavenumber dependent chi pairs indexed by tuple pairs
    
    '''
    
    assert PRISM.rank>1,'the chi calculation is only valid for multicomponent systems'
    
    if PRISM.directCorr.space == Space.Real:
        PRISM.domain.MatrixArray_to_real(PRISM.direcCorr)
        
    totalDensity = 0.
    for i,t1 in enumerate(PRISM.types):
        totalDensity += PRISM.siteDensityMatrix[i,i]
    
    chi = {}
    for i,t1 in enumerate(PRISM.types):
        for j,t2 in enumerate(PRISM.types):
            if i<j:
                C_AA = PRISM.directCorr[i,i]
                C_AB = PRISM.directCorr[i,j]
                C_BB = PRISM.directCorr[j,j]
                
                chi[t1,t2] = chi[t2,t1] = 0.5 * totalDensity * (C_AA + C_BB - 2*C_AB)
                
    
    
    return chi

Overwriting ../pyPRISM/calculate/prism/chi.py


## TESTING

In [10]:
%%run_and_write ../pyPRISM/test/CalcPRISM_TestCase.py
#!python
from __future__ import division,print_function
import pyPRISM
import numpy as np
import unittest

class CalcPRISM_TestCase(unittest.TestCase):
    def setup(self):
        '''Construct a simple but fully specified PRISM problem'''
        
        sys = pyPRISM.System(['A','B'])
        
        sys.domain = pyPRISM.Domain(dr=0.1,length=512)
        
        sys.density['A'] = 0.5
        sys.density['B'] = 0.5
        
        sys.closure.setUnset(pyPRISM.closure.PercusYevick())
        
        sys.potential.setUnset(pyPRISM.potential.HardSphere(sigma=1.0))
        
        sys.intraMolCorr['A','A'] = pyPRISM.intraMolCorr.SingleSite()
        sys.intraMolCorr['A','B'] = pyPRISM.intraMolCorr.NoIntra()
        sys.intraMolCorr['B','B'] = pyPRISM.intraMolCorr.Gaussian(sigma=1.0,length=10)
        
        PRISM = sys.createPRISM()
        
        return PRISM

    def test_pair_correlation(self):
        '''Can we calculate pair_correlations?'''
        PRISM = self.setup()
        PRISM.solve(disp=False)
        result = pyPRISM.calculate.prism.pair_correlation(PRISM)
        
    def test_structure_factor(self):
        '''Can we calculate structure factors?'''
        PRISM = self.setup()
        PRISM.solve(disp=False)
        result = pyPRISM.calculate.prism.structure_factor(PRISM)
        
    def test_pmf(self):
        '''Can we calculate pmf's?'''
        PRISM = self.setup()
        PRISM.solve(disp=False)
        result = pyPRISM.calculate.prism.pmf(PRISM)
        
        


Overwriting ../pyPRISM/test/CalcPRISM_TestCase.py


## RUN ALL

In [11]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(CalcPRISM_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_pair_correlation (__main__.CalcPRISM_TestCase)
Can we calculate pair_correlations? ... ok
test_pmf (__main__.CalcPRISM_TestCase)
Can we calculate pmf's? ... ok
test_structure_factor (__main__.CalcPRISM_TestCase)
Can we calculate structure factors? ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.326s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>